In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import folium

In [ ]:
# 1. Загрузка данных
stops = gpd.read_file("/Users/user/PycharmProjects/urban-graph-forecasting/data/train_dataset_codd_train/Векторные данные/Остановки_ОТ.shp")  # файл с остановками
roads = gpd.read_file("/Users/user/PycharmProjects/urban-graph-forecasting/data/train_dataset_codd_train/Векторные данные/Streets_исходные.shp")  # файл с дорогами

In [ ]:
stops 

In [ ]:
roads

In [ ]:
# Преобразуем данные в проекцию, которая использует метры (например, EPSG:32633)
stops = stops.to_crs(epsg=32633)
roads = roads.to_crs(epsg=32633)

# Фильтрация дорог, где колонка 'Foot' равна 1 (пешеходные дороги)
roads_filtered = roads[roads['Foot'] == 1]

# Преобразуем обратно в EPSG:4326 для отображения на карте
roads_filtered = roads_filtered.to_crs(epsg=4326)
stops = stops.to_crs(epsg=4326)

# Создание карты на основе координат первой остановки
m = folium.Map(location=[stops.geometry.y.mean(), stops.geometry.x.mean()], zoom_start=12)

# Добавление пешеходных дорог на карту
for _, road in roads_filtered.iterrows():
    # Для каждой дороги добавляем линию
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in road.geometry.coords], 
        color='blue', weight=2.5, opacity=1
    ).add_to(m)

# Добавление остановок на карту
for _, stop in stops.iterrows():
    folium.Marker([stop.geometry.y, stop.geometry.x], popup=f"Stop ID: {stop['TrStopId']}").add_to(m)

# Сохранить карту в файл
m.save('map_with_roads_and_stops.html')

# Печать завершения
print("Карта сохранена в 'map_with_roads_and_stops.html'.")

In [ ]:
m

In [ ]:
import momepy
from momepy.datasets import get_path
import geopandas as gpd
import networkx as nx

G = momepy.gdf_to_nx(roads_filtered, multigraph=False)



In [ ]:
nx.number_connected_components(G)

In [ ]:
component_sizes = [len(component) for component in nx.connected_components(G)]

print("Количество вершин в каждой связной компоненте:", component_sizes)

In [ ]:
# Находим все связные компоненты
components = list(nx.connected_components(G))

# Находим самую крупную компоненту
largest_component = max(components, key=len)

# Создаем новый граф, содержащий только вершины и рёбра самой крупной компоненты
G_largest = G.subgraph(largest_component).copy()